In [156]:
# Imports
import requests
import os

from IPython.display import HTML
from ipywidgets import interact

# Functions
def multireddit(subreddits, limit=5, orderby='score'):
    ''' Displays an HTML listing of the top `limit` articles from the
    various `subreddits` specified by the user.  These articles will be
    sorted by the field specified by `orderby`

      - `subreddits` is a comma-separated list of subreddits (ie.
        `linux,linuxmasterrace`)

      - `orderby` is any field in the Reddit JSON data for a particular
        article (ie. `score`, `title`, `created_utc`, etc.)
    '''

    # Process function arguments
    if not subreddits:
        return
    
    subList = subreddits.split(',')
    
    # TODO: Fetch subreddit data
    articles = []
    for sub in subList:
        headers  = {'user-agent': 'reddit-{}'.format(os.environ['USER'])}
        # tell user if the subreddit they typed doesn't exist
        try:
            # gets past reddit anti-bot stuff
            while True:
                response = requests.get('https://www.reddit.com/r/{}/.json'.format(sub), headers=headers)
                data = response.json()
                articles += data['data']['children']
                if response.status_code == 200:
                    break
        except:
            if(sub != ""):
                print("Subreddit {} does not exist, cannot display".format(sub))
                return
    
    
    # TODO: Sort and limit data, and construct HTML results
    html = '<table><tbody>'
    
    # if an invalid subreddit somehow has a url, tell the user it is invalid
    try:
        # Handles sorting by title correctly
        if orderby == 'title':
            for index, article in enumerate(sorted(articles, key=lambda a: a['data'][orderby].lower())[:limit]):
                html += '''
                <tr>
                    <td>{}</td>
                    <td style="text-align: left"><a href="{}" target="_blank">{}</td>
                    <td>{}</td>
                    <td><a href="https://reddit.com{}" target="_blank">Comments</td>
                </tr>
                '''.format(index+1, article['data']['url'], article['data']['title'], article['data']['score'], article['data']['permalink'])
        else:
            for index, article in enumerate(sorted(articles, key=lambda a: a['data'][orderby])[:limit]):
                html += '''
                <tr>
                    <td>{}</td>
                    <td style="text-align: left"><a href="{}" target="_blank">{}</td>
                    <td>{}</td>
                    <td><a href="https://reddit.com{}" target="_blank">Comments</td>
                </tr>
                '''.format(index+1, article['data']['url'], article['data']['title'], article['data']['score'], article['data']['permalink'])
    except:
        print("One of your subreddits is invalid, cannot display.")
    
    html += '</tbody></table>'

    display(HTML(html))

In [157]:
ORDERBY = {
    'Score': 'score',
    'Creation Time': 'created_utc',
    'Title': 'title'
}

interact(multireddit, subreddits='', limit=(1, 20), orderby=ORDERBY)

A Jupyter Widget

<function __main__.multireddit>

# Questions

1. I parsed the subreddit argument by using the split function delimited by a comma. This converted the string into a list of subreddits that was easily handle using a for loop. 
2. I fetched the JSON data by using the subList with a loop. In the loop, the subreddit was placed into an already formatted URL using the .format() function and this URL was accessed with the requests.get() function, which returned JSON data. I used a for loop to iterate over each entry in the JSON. I then concatenated the data for each new subreddit.
3. I constructed the formatted HTML by using table and tbody elements, then using tr for each row. I put each tr in a for loop, and then accessed each link's data using `article['data']['url']`,  `article['data']['title']`, `article['data']['score']`, and  `article['data']['permalink']`. I used the .format() function to add this data into my formatted list. I then used the display() function to display the resulting HTML code, which was processed by my web browser.